In [17]:
from tensorflow.keras.applications import VGG16

# include_top : Classifier 사용여부
model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150,150,3))

model_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [12]:
# dogs and cats training data set에 대한 feature map을 추출하기
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '../data/cat_dog_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

datagen = ImageDataGenerator(rescale=1/255)
batch_size = 20

def extract_feature(directory, sample_count):
    # 추출한 feature map을 담을 ndarray
    features = np.zeros(shape=(sample_count,4,4,512))
    labels = np.zeros(shape=(sample_count,))
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary'
    )
    
    i = 0
    
    for x_data_batch, t_data_batch in generator:
        feature_batch = model_base.predict(x_data_batch)
        features[i*batch_size:(i+1)*batch_size] = feature_batch
        labels[i*batch_size:(i+1)*batch_size] = t_data_batch
        
        i += 1
        if i * batch_size >= sample_count:
            break
    
    return features, labels

In [13]:
trian_features, train_labels = extract_feature(train_dir, 2000)
validation_features, validation_labels = extract_feature(validation_dir, 1000)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [36]:
trian_features = np.reshape(trian_features, (2000,4*4*512))
validation_features = np.reshape(validation_features, (1000,4*4*512))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(units=256, activation='relu', input_shape=(4*4*512,)))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
history = model.fit(x=trian_features, y=train_labels, epochs=30, batch_size=64,
                    validation_data=(validation_features, validation_labels),
                    verbose=1)

Epoch 1/30
32/32 [==============================] - 0s 6ms/step - loss: 0.5408 - accuracy: 0.7105 - val_loss: 0.3706 - val_accuracy: 0.8390
Epoch 2/30
32/32 [==============================] - 0s 4ms/step - loss: 0.3321 - accuracy: 0.8580 - val_loss: 0.2937 - val_accuracy: 0.8870
Epoch 3/30
32/32 [==============================] - 0s 4ms/step - loss: 0.2843 - accuracy: 0.8795 - val_loss: 0.2833 - val_accuracy: 0.8840
Epoch 4/30
32/32 [==============================] - 0s 4ms/step - loss: 0.2342 - accuracy: 0.9115 - val_loss: 0.2610 - val_accuracy: 0.8930
Epoch 5/30
32/32 [==============================] - ETA: 0s - loss: 0.2203 - accuracy: 0.91 - 0s 4ms/step - loss: 0.2180 - accuracy: 0.9185 - val_loss: 0.2610 - val_accuracy: 0.9000
Epoch 6/30
32/32 [==============================] - 0s 4ms/step - loss: 0.1979 - accuracy: 0.9290 - val_loss: 0.2568 - val_accuracy: 0.8950
Epoch 7/30
32/32 [==============================] - 0s 4ms/step - loss: 0.1636 - accuracy: 0.9435 - val_loss: 0.2456 -

- 내가 짠 코드

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=1024, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-6), loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
history = model.fit(x=trian_features, y=train_labels, epochs=50, 
                    validation_data=(validation_features, validation_labels),
                    verbose=1)

Epoch 1/50
63/63 [==============================] - 0s 5ms/step - loss: 0.3094 - accuracy: 0.8775 - val_loss: 0.3260 - val_accuracy: 0.8750
Epoch 2/50
63/63 [==============================] - 0s 5ms/step - loss: 0.3041 - accuracy: 0.8895 - val_loss: 0.3210 - val_accuracy: 0.8770
Epoch 3/50
63/63 [==============================] - 0s 5ms/step - loss: 0.2976 - accuracy: 0.8880 - val_loss: 0.3161 - val_accuracy: 0.8830
Epoch 4/50
63/63 [==============================] - 0s 5ms/step - loss: 0.2896 - accuracy: 0.8960 - val_loss: 0.3127 - val_accuracy: 0.8810
Epoch 5/50
63/63 [==============================] - 0s 5ms/step - loss: 0.2855 - accuracy: 0.8985 - val_loss: 0.3094 - val_accuracy: 0.8800
Epoch 6/50
63/63 [==============================] - 0s 5ms/step - loss: 0.2804 - accuracy: 0.9030 - val_loss: 0.3046 - val_accuracy: 0.8850
Epoch 7/50
63/63 [==============================] - 0s 5ms/step - loss: 0.2810 - accuracy: 0.8935 - val_loss: 0.3023 - val_accuracy: 0.8840
Epoch 8/50
63/63 [==